# Classifieur de Spam

lien du brief : https://simplonline.co/briefs/97a4822f-8af0-4607-86b3-83dbfdd05d5e 

## Descriptif de Simplonline

#### Contexte :

Concevoir un classifieur de détection automatique de SPAM.

La collection SMS Spam est un ensemble de messages SMS marqués qui ont été collectés pour la recherche sur les SMS Spam. Elle contient un ensemble de messages SMS en anglais de 5 574 messages, étiquetés selon qu'ils sont ham (légitimes) ou spam.
Je vous encourage à vous documenter sur les caractéristiques type des spam et de développer votre stratégie de préparation des données dans ce sens.

En tant que développeur IA, voici les missions :
- Analyse du besoin
- Construction d'un pipeline de ML
- Prétraitement des données
- Entrainement, fine tuning, validation et sélection d'un modèle de classification

Les fichiers contiennent un message par ligne. Chaque ligne est composée de deux colonnes : v1 contient le label (ham ou spam) et v2 contient le texte brut.

liens :

dataset : https://github.com/remijul/dataset/blob/master/SMSSpamCollection

informations : https://archive.ics.uci.edu/dataset/228/sms+spam+collection 

#### Critères de performance :

- compréhension du jeux de données
- capacité à préparer les données
- performance des modèles de prédiction
- capacité à apporter une solution dans le temps imparti
- rédaction du notebook
- qualité du synthèse du travail

#### Livrables :

* créer un/des notebook reproductible, commenté, expliqué (IMPORTANT !)
* créer un repo git et un espace sur github/gitlab pour le projet (code refactorisé)
* faire une présentation (slides) qui explique votre démarche et les résultats obtenus avec :
- un document technique qui explique l'outil
- la procédure suivie pour préparer les données et le preprocessing
- la procédure suivie pour trouver un modèle adapté
- le modèle d'IA sélectionné

BONUS :
* Application streamlit qui fait de la prédiction en temps réel d'un message déposé par l'utilisateur


### Analyse du contexte 

#### D'où viennent les données : Par qui ? Pour quoi ? Comment ?


SMS Spam Collection est un ensemble public de messages étiquetés par SMS qui ont été collectés pour la recherche sur le spam pour les téléphones portables.

##### Instances = 5574

##### Informations supplémentaires

Ce corpus a été collecté à partir de sources de recherche gratuites ou gratuites sur Internet:

Une collection de 425 messages de spam par SMS a été extraite manuellement du site Web de Grumbletext. Il s'agit d'un forum britannique dans lequel les utilisateurs de téléphones portables font des déclarations publiques sur les SMS spam, la plupart d'entre eux sans signaler le message de spam reçu. L'identification du texte des messages de spam dans les revendications est une tâche très difficile et longue, et il a consisté à numériser soigneusement des centaines de pages Web. Le site Web de Grumbletext est le suivant: http://www.grumbletext.co.uk/.
Un sous-ensemble de 3 375 SMS choisis au hasard par jambon du NUS SMS Corpus (NSC), qui est un ensemble de données d'environ 10 000 messages légitimes collectés pour la recherche au Département de l'informatique de l'Université nationale de Singapour. Les messages proviennent en grande partie de Singapouriens et principalement d'étudiants fréquentant l'Université. Ces messages ont été recueillis auprès de volontaires qui ont été informés que leurs contributions allaient être rendues publiques. Le NUS SMS Corpus est disponible à l'adresse suivante: http://www.comp.nus.edu.sg/.rpnlpir/downloads/corpora/smsCorpus/.
Une liste de 450 SMS de type jambon collectés sur la thèse de doctorat de Caroline Tag disponible à l'adresse http://etheses.bham.ac.uk/253/1/Tagg09PhD.pdf.
Enfin, nous avons incorporé le SMS Spam Corpus v.0.1 Big. Il contient 1 002 messages de mja SMS et 322 messages de spam et il est disponible en public à l'adresse suivante: http://www.esp.uem.es/jmgomez/smsspamcorpus/. Ce corpus a été utilisé dans les recherches universitaires suivantes:

1 G-3mez Hidalgo, J.M., Cajigas Bringas, G., Puertas Sanz, E., Carrero Garcia, F. Filtration par SMS basée sur le contenu. Actes du Colloque 2006 de l'ACM sur l'ingénierie des documents (ACM DOCENG'06), Amsterdam (Pays-Bas), 10-13, 2006.

Cormack, G. V., G-3mez Hidalgo, J. M., et Puertas Sonz, E. Ingénierie technique pour filtrage de spam mobile (SMS).  Actes de la trentième Conférence internationale annuelle de la CMA sur la recherche et le développement dans la recherche et le développement dans le domaine de la recherche et de l'information (ACM SIGIR'07), New York, NY, 871-872, 2007.

3 Cormack, G. V., G-3mez Hidalgo, J. M., et Puertas Sonz, E. Filtration de spam pour les messages courts. Actes de la seizième Conférence de l'ACM sur la gestion de l'information et des connaissances (ACM CIKM'07). Lisbonne, Portugal, 313-320, 2007.

##### Des valeurs manquantes ont-elles été des valeurs?

Non



#### A quoi on reconnait un Spam ?

- Généralement, les messages malveillants sont envoyés à destination d'un grand nombre de cibles, ils ne sont pas ou peu personnalisés.

- Le message évoque un dossier, une facture, un thème qui ne vous parle pas ? Il s'agit certainement d'un courriel malveillant.

(source : https://www.economie.gouv.fr/entreprises/comment-lutter-contre-spams)

#### Comment faire pour reconnaitre un Spam à partir d'un texte ? (hypotèse de travail)

rechercher dans le texte brut :
- des mots clé comme : 'URGENT!', 'Quiz!', 'YOU!', 'Txt:', 'now!', 'Call ', 'Win', 'WINNER', '!!', 
- des montions à de l'argent
- des numéros de téléphone
- des e-mails
- des liens
- utilisation de mot en majuscule

## Importation des modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, MultinomialNB
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

## Pré-processing


#### Ouverture du fichier

df_git : nom de la data frame extrait du fichier

In [2]:
df_git = pd.read_csv('https://raw.githubusercontent.com/remijul/dataset/master/SMSSpamCollection', 
                 sep='\t',on_bad_lines='skip', header=None)
df_git.head()

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#### mise en forme des données 

la colone '0' = 'classification'

la colone '1' = 'sms' 


In [3]:
df_git.rename(columns={0:'classification'}, inplace=True)
df_git.rename(columns={1:'sms'}, inplace=True)
df_git.head()

,classification,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#### Nettoyage des données

Les données nan dans la colone ham impliquent que nous ne pouvons pas dire à la machine la classe du texte, on peut les metres de coté pour une mise en application après amélioration du model.

Les données nan dans la colone 'texte' impliquent que nous n'avons pas le texte à évaluer. Nous allons suprimer ces lignes car il n'y a rien à y traiter.

In [4]:
df_git['classification'].isna().value_counts()

classification
False    5572
Name: count, dtype: int64

nous observons :

    classification
    False    5572    

il n'y a pas de valeur na dans cette colone

In [5]:
df_git['sms'].isna().value_counts()

sms
False    5572
Name: count, dtype: int64

nous observons :

    sms
    False    5572   

il n'y a pas de valeur na dans cette colone

In [6]:
df_git.duplicated().value_counts()

False    5169
True      403
Name: count, dtype: int64

nous avons pour résultats :

    False    5169
    True      403

donc il y a 403 doublons que nous allons supprimer

In [7]:
df_git = df_git.drop_duplicates()

In [8]:
df_git

,classification,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


#### Encodage

 => création d'un dico et mapping : 0:'ham', 1:'spam'

In [9]:
label_encod = LabelEncoder()
df_git['classification'] = label_encod.fit_transform(df_git['classification'])
df_git

,classification,sms
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will ü b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


martice : présence (1) ou absence (0) de certains éléments dans le texte

In [10]:
mot_cles = ['URGENT!', 'Quiz!', 'YOU!', 'Txt:', 'now!', 'Call ', 'Win', 'WINNER', '!!']
df_git['mot_cles'] = df_git['sms'].str.contains('|'.join(mot_cles), case=False)
df_git['mot_cles'].value_counts()


mot_cles
False    4475
True      694
Name: count, dtype: int64

In [11]:
mot_cles = ['£', '€', '\$']
df_git['argent'] = df_git['sms'].str.contains('|'.join(mot_cles), case=False)
df_git['argent'].value_counts()

argent
False    4933
True      236
Name: count, dtype: int64

In [12]:
def no_tel_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : boolean
    ---------------------
    je crée le pattern des numeros de tel
    je recherche dans une chaine de caractère si je trouve le pattern    
    """
    pattern = r"\+?\d{1,3}[-.\s]?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,4}"
    numeros = re.findall(pattern, sms)
    return bool(numeros)

In [13]:
df_git['telephone'] = df_git['sms'].apply(no_tel_posible)
df_git['telephone'].value_counts()

telephone
False    4615
True      554
Name: count, dtype: int64

In [14]:
def email_posible (sms) :
    """
    entrée : chaine de caractère
    sortie : boolean
    ---------------------
    je crée le pattern des numeros de tel
    je recherche dans une chaine de caractère si je trouve le pattern    
    """
    pattern = r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+"
    match = re.findall(pattern, sms)
    return bool(match)


In [15]:
df_git['email'] = df_git['sms'].apply(email_posible)
df_git['email'].value_counts()

email
False    5162
True        7
Name: count, dtype: int64

In [16]:
mot_cles_lien = ['http', 'https', 'www.', 'click here']
df_git['lien'] = df_git['sms'].str.contains('|'.join(mot_cles_lien), case=False)
df_git['lien'].value_counts()

lien
False    5076
True       93
Name: count, dtype: int64

In [17]:
def mot_maj_posible (sms) :
    """
    entrée : chaine de carractaire
    sortie : boolean
    ---------------------
    je recherche dans une chaine de caractère si je trouve 2 majuscules à la suite   
    """
    pattern = "[A-Z]{3}"
    match = re.findall(pattern, sms)
    return bool(match)

In [18]:
df_git['maj'] = df_git['sms'].apply(mot_maj_posible)
df_git['maj'].value_counts()

maj
False    4405
True      764
Name: count, dtype: int64

In [19]:
df_git

,classification,sms,mot_cles,argent,telephone,email,lien,maj
0,0,"Go until jurong point, crazy.. Available only ...",False,False,False,False,False,False
1,0,Ok lar... Joking wif u oni...,False,False,False,False,False,False
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,True,False,True,False,False,False
3,0,U dun say so early hor... U c already then say...,False,False,False,False,False,False
4,0,"Nah I don't think he goes to usf, he lives aro...",False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...,True,True,True,False,False,True
5568,0,Will ü b going to esplanade fr home?,False,False,False,False,False,False
5569,0,"Pity, * was in mood for that. So...any other s...",False,False,False,False,False,False
5570,0,The guy did some bitching but I acted like i'd...,False,False,False,False,False,False


#### Création des df : df_train et df_test

Les données récupérées sont séparées en 2 df :

- df_train : contient 80 % des données, les models sont entrainés grâce à ce jeu de données.
- df_test : contient 20 % des données, les performances des models seront testées grâce à ce jeu de données.

In [20]:
trainSet, testSet = train_test_split(df_git, test_size=0.2, random_state=99, stratify=df_git['classification'])

In [21]:
trainSet.drop('sms', axis=1, inplace=True)
testSet.drop('sms', axis=1, inplace=True)

In [22]:
y_train = trainSet['classification']
y_train.value_counts()

classification
0    3613
1     522
Name: count, dtype: int64

In [23]:
y_test = testSet['classification']
y_test.value_counts()

classification
0    903
1    131
Name: count, dtype: int64

In [24]:
x_train = trainSet
x_train.drop('classification', axis=1, inplace=True)
x_train

,mot_cles,argent,telephone,email,lien,maj
3104,False,False,False,False,False,False
4160,False,False,False,False,False,False
675,False,False,False,False,False,False
3448,False,False,False,False,False,False
3597,False,False,False,False,False,False
...,...,...,...,...,...,...
4145,False,False,False,False,False,False
296,True,False,True,False,False,True
484,False,False,False,False,False,False
2349,False,False,False,False,False,False


In [25]:
x_test = testSet
x_test.drop('classification', axis=1, inplace=True)
x_test

,mot_cles,argent,telephone,email,lien,maj
1867,False,False,False,False,False,False
2850,True,False,True,False,False,True
4715,False,False,False,False,False,False
212,False,False,False,False,False,False
5294,True,True,True,False,True,True
...,...,...,...,...,...,...
5547,False,False,False,False,False,True
2986,False,False,False,False,False,False
4806,False,False,False,False,False,False
3655,False,False,False,False,False,False


## Modélisation


#### choix du model

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

Selon nos données et l'arbre de décision de scikit-learn :
- nous cherchons à prédir une classe et avons plus de 50 données,
- nos classes sont déjà définies et avons moins de 100 000 données
- nos données sont au format 'string'

nous sommes donc dans un cas d'utilisation du model : naive bayers

#### Création du model

In [26]:
#model = MultinomialNB()

model = MultinomialNB (force_alpha=True)

#### Evaluation du model

In [27]:
def evaluation (model) :
    """
    entrée : model
    sortie : matrice de confusion et classification_report
    -----------------------------
    j'entraine un model, sur les x_train et y_train
    je calcule les prédictions du model
    """
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    score = accuracy_score(y_test, y_pred)
    print (score)
    print (confusion_matrix(y_test, y_pred))
    print (classification_report(y_test, y_pred))



In [28]:
evaluation (model)

0.9342359767891683
[[900   3]
 [ 65  66]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       903
           1       0.96      0.50      0.66       131

    accuracy                           0.93      1034
   macro avg       0.94      0.75      0.81      1034
weighted avg       0.94      0.93      0.93      1034



###### Notre model a un score moyen de 93%

Nous obtenons 3 'ham' sur 903 et 65 'spam' sur 131 mal catégorisé, 
soit 93 % et 50 % de précition réspective


notre model est donc plutot bon pour un premier test

#### Diagnostique

Pour améliorer le model, nous allons mettre en place une pipeligne
nous allons utiliser d'autres models et d'autres encodages